import useful packages

In [1]:
import sys
import argparse
import pandas as pd
import numpy as np
import io
import os

definition of useful helper function

In [8]:
'''
Defining functions below
'''

def store_vcf_header(path):
    '''
    input: a string path to the vcf file
    output: a list of lines starting with '##'

    '''
    with open(path, 'r') as f:
        lines=[l for l in f if l.startswith('##')]
    return lines

def read_vcf(path):
    '''
    input: a string path to the vcf file
    output: a pandas dataframe of the vcf file
    read_vcf from https://gist.github.com/dceoy/99d976a2c01e7f0ba1c813778f9db744
    '''
    with open(path, 'r') as f:
        lines=[l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})

def sample_heterozygous(df):
    '''
    input: dataframe from a vcf file, dataframe should be one sample's fields. can be made by:
        df=vcf_df['sample_name'].str.split(":", expand=True)
    output: boolean list denoting which loci are heterozygous (this ignores any "2" or "3" alt alleles)

    '''
    heterozygous=((df[0] == '0|1') | (df[0] == '0/1')| (df[0] == '1/1') | (df[0] == '1|1'))
    return heterozygous

def sample_not_alt1(df):
    '''
    input: dataframe from a vcf file, dataframe should be one sample's fields. can be made by:
        df=vcf_df['sample_name'].str.split(":", expand=True)
    output: boolean list denoting which loci are homozygous for the reference allele

    '''
    not_alt=((df[0] == '0|0') | (df[0] == '0/0'))
    return not_alt

def het_verify(df):
    '''
    input: dataframe from a vcf file, dataframe should be one sample's fields. can be made by:
        df=vcf_df['sample_name'].str.split(":", expand=True)
        Expected input has already been filtered down to just heterozygous alleles.
    output: boolean list denoting which loci are heterozygous, and fall within expected ranges of allele ratios, and have at least 8 variant reads

    '''

    allele_depth=df[1]
    allele_depth=allele_depth.str.split(',', expand=True).iloc[:,0:2]
    vrf=allele_depth[1].astype(int)/np.sum(allele_depth.astype(int), axis=1)
    verify=( (vrf > 0.35) & (allele_depth[1].astype(int) > 8) )
    return verify

def filter(sample, control):
    '''
    input: dataframes of progeny and parental vcf data, dataframes should each be one sample's fields. can be made by:
        df=vcf_df['sample_name'].str.split(":", expand=True)
    output: subsetted dataframe with just the rows in the progeny sample that are heterozygous, but are putatively homozygous for the reference allele in control
    '''
    
    sample_het_bool=sample_heterozygous(sample)
    sample_sub=sample[sample_het_bool]
    sample_sub=sample_sub[het_verify(sample_sub)]
    
    control_sub=control.loc[sample_sub.index,:]
    control_filter=sample_not_alt1(control_sub)

    sample_sub=sample_sub[(control_filter)]
    return(sample_sub)

analysis of enAs cohort vcf file

In [6]:
#readin vcf file 
NK_df=read_vcf('NKGOF_indels_filtered.vcf')
NK_header=store_vcf_header('NKGOF_indels_filtered.vcf')

#filter variants based on quality metrics (only keep the ones on Chr1-22, X)
NK_df=NK_df[(([len(i) < 7 for i in NK_df['CHROM']]) & (NK_df['FILTER'] == 'PASS') & (NK_df['QUAL'].astype(float) > 1000))] 
NK_df



,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,Donor0958_NT,Donor0958_OR7A10OE,Donor0958_OR7A10Stop
6,chr1,820905,.,ATCCACCCTGTCTACACTACCTGCTTGTCCAGCAGG,A,1235.93,PASS,AC=3;AF=0.500;AN=6;BaseQRankSum=0.436;DP=76;Ex...,GT:AD:DP:GQ:PL,"0/1:10,10:20:99:389,0,383","0/1:17,11:28:99:409,0,683","0/1:3,11:14:92:448,0,92"
10,chr1,828389,.,AT,A,1004.93,PASS,AC=3;AF=0.500;AN=6;BaseQRankSum=1.09;DP=99;Exc...,GT:AD:DP:GQ:PL,"0/1:14,16:30:99:361,0,304","0/1:25,20:45:99:429,0,568","0/1:9,10:19:99:225,0,196"
11,chr1,832736,.,AGTTTT,A,1173.93,PASS,AC=3;AF=0.500;AN=6;BaseQRankSum=-5.100e-02;DP=...,GT:AD:DP:GQ:PL,"0/1:18,6:24:99:198,0,737","0/1:11,16:27:99:635,0,406","0/1:9,9:18:99:351,0,350"
12,chr1,833758,.,CAT,C,2265.93,PASS,AC=3;AF=0.500;AN=6;BaseQRankSum=0.905;DP=112;E...,GT:AD:DP:GQ:PL,"0/1:11,16:27:99:637,0,414","0/1:19,27:46:99:1071,0,712","0/1:20,15:35:99:568,0,795"
16,chr1,840409,.,TAA,"T,TAAAAA",2344.67,PASS,"AC=3,3;AF=0.500,0.500;AN=6;BaseQRankSum=-5.070...",GT:AD:DP:GQ:PL,"1/2:1,15,6:22:99:719,232,214,524,0,609","1/2:0,17,13:30:99:997,495,515,561,0,638","1/2:0,9,9:18:99:651,363,351,321,0,351"
...,...,...,...,...,...,...,...,...,...,...,...,...
913760,chrY,11788211,.,A,AC,1002.09,PASS,AC=6;AF=1.00;AN=6;DP=23;ExcessHet=0.0000;FS=0....,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,8:8:24:1|1:11788211_A_AC:346,24,0:11788211","1|1:0,12:12:36:1|1:11788211_A_AC:535,36,0:1178...","1|1:0,3:3:9:1|1:11788211_A_AC:135,9,0:11788211"
913769,chrY,26403180,.,GC,G,3091.69,PASS,AC=6;AF=1.00;AN=6;DP=75;ExcessHet=0.0000;FS=0....,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,28:28:84:1|1:26403180_GC_G:1260,84,0:264...","1|1:0,23:23:69:1|1:26403180_GC_G:1035,69,0:264...","1|1:0,18:18:54:1|1:26403180_GC_G:810,54,0:2640..."
913770,chrY,26403183,.,AAAGGGAGGGCAGGGCC,A,3091.69,PASS,AC=6;AF=1.00;AN=6;DP=77;ExcessHet=0.0000;FS=0....,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,28:28:84:1|1:26403180_GC_G:1260,84,0:264...","1|1:0,23:23:69:1|1:26403180_GC_G:1035,69,0:264...","1|1:0,18:18:54:1|1:26403180_GC_G:810,54,0:2640..."
913771,chrM,302,.,A,AC,39840.69,PASS,AC=6;AF=1.00;AN=6;BaseQRankSum=0.750;DP=1893;E...,GT:AD:DP:GQ:PL,"1/1:57,815:921:99:22746,284,0","1/1:7,287:305:99:8518,653,0","1/1:2,277:303:99:8590,854,0"


In [7]:
#split into LSL and enAs
NT=NK_df['Donor0958_NT'].str.split(":", expand=True)
OE=NK_df['Donor0958_OR7A10OE'].str.split(":", expand=True)
Stop=NK_df['Donor0958_OR7A10Stop'].str.split(":", expand=True)
Stop 

,0,1,2,3,4,5,6,7
6,0/1,"3,11",14,92,"448,0,92",None,None,None
10,0/1,"9,10",19,99,"225,0,196",None,None,None
11,0/1,"9,9",18,99,"351,0,350",None,None,None
12,0/1,"20,15",35,99,"568,0,795",None,None,None
16,1/2,"0,9,9",18,99,"651,363,351,321,0,351",None,None,None
...,...,...,...,...,...,...,...,...
913760,1|1,"0,3",3,9,1|1,11788211_A_AC,"135,9,0",11788211
913769,1|1,"0,18",18,54,1|1,26403180_GC_G,"810,54,0",26403180
913770,1|1,"0,18",18,54,1|1,26403180_GC_G,"810,54,0",26403180
913771,1/1,"2,277",303,99,"8590,854,0",None,None,None


In [11]:
#filter variants unique to control 
OE_filtered=NK_df.loc[filter(OE, NT).index,:]
Stop_filtered=NK_df.loc[filter(Stop, NT).index,:]
OE_filtered

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,Donor0958_NT,Donor0958_OR7A10OE,Donor0958_OR7A10Stop
652,chr1,2348720,.,G,GGCACGCACACACAC,1107.12,PASS,AC=2;AF=0.333;AN=6;BaseQRankSum=2.12;DP=65;Exc...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0/0:17,0:17:38:.:.:0,38,631","0|1:7,14:21:99:0|1:2348700_C_T:567,0,232:2348700","0|1:11,14:25:99:0|1:2348700_C_T:551,0,420:2348700"
977,chr1,3640485,.,ACCCGAGGCTCTGAGCATCAGCAGGGCGGGGTGCAGCG,A,1684.92,PASS,AC=4;AF=0.667;AN=6;DP=85;ExcessHet=0.0000;FS=0...,GT:AD:DP:GQ:PL,"0/0:19,0:19:48:0,48,720","1/1:0,25:25:79:1122,79,0","1/1:0,13:13:42:582,42,0"
1431,chr1,4616480,.,G,GC,1521.12,PASS,AC=2;AF=0.333;AN=6;BaseQRankSum=0.391;DP=95;Ex...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0/0:21,0:21:60:.:.:0,60,900","0|1:14,23:37:99:0|1:4616480_G_GC:895,0,514:461...","0|1:20,17:37:99:0|1:4616480_G_GC:637,0,769:461..."
5690,chr1,15614193,.,CTTTTTTTTTTTT,C,1150.39,PASS,AC=2;AF=0.333;AN=6;BaseQRankSum=1.88;DP=61;Exc...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0/0:13,0:13:10:.:.:0,10,460","0|1:9,17:26:99:0|1:15614180_T_C:687,0,323:1561...","0|1:10,12:22:99:0|1:15614180_T_C:474,0,384:156..."
10719,chr1,30644882,.,TTTGA,T,1015.12,PASS,AC=2;AF=0.333;AN=6;BaseQRankSum=0.096;DP=70;Ex...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0/0:21,0:21:51:.:.:0,51,765","0|1:14,16:30:99:0|1:30644882_TTTGA_T:630,0,524...","0|1:8,10:18:99:0|1:30644882_TTTGA_T:396,0,306:..."
...,...,...,...,...,...,...,...,...,...,...,...,...
910060,chrX,141401882,.,ATATATATATGTGTATATATATATATG,A,1072.46,PASS,AC=4;AF=0.667;AN=6;DP=27;ExcessHet=0.0000;FS=0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0/0:2,0:2:6:.:.:0,6,74","1|1:0,16:16:48:1|1:141401882_ATATATATATGTGTATA...","1|1:0,9:9:27:1|1:141401882_ATATATATATGTGTATATA..."
910061,chrX,141401950,.,GTA,G,1198.79,PASS,AC=4;AF=0.667;AN=6;DP=29;ExcessHet=0.0000;FS=0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0/0:2,0:2:3:.:.:0,3,45","1|1:0,15:15:45:1|1:141401882_ATATATATATGTGTATA...","1|1:0,12:12:36:1|1:141401882_ATATATATATGTGTATA..."
910063,chrX,141402038,.,GTATATATATATA,G,1591.97,PASS,AC=4;AF=0.667;AN=6;DP=44;ExcessHet=0.0000;FS=0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0/0:8,0:8:21:.:.:0,21,315","1|1:0,21:21:63:1|1:141401882_ATATATATATGTGTATA...","1|1:0,15:15:45:1|1:141401882_ATATATATATGTGTATA..."
912790,chrX,152083224,.,AC,A,1369.92,PASS,AC=4;AF=0.667;AN=6;DP=60;ExcessHet=0.0000;FS=0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0/0:25,0:25:40:.:.:0,40,924","1|1:0,18:18:54:1|1:152083205_T_TA:727,54,0:152...","1|1:0,16:16:48:1|1:152083205_T_TA:662,48,0:152..."


In [12]:
Stop_filtered

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,Donor0958_NT,Donor0958_OR7A10OE,Donor0958_OR7A10Stop
652,chr1,2348720,.,G,GGCACGCACACACAC,1107.12,PASS,AC=2;AF=0.333;AN=6;BaseQRankSum=2.12;DP=65;Exc...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0/0:17,0:17:38:.:.:0,38,631","0|1:7,14:21:99:0|1:2348700_C_T:567,0,232:2348700","0|1:11,14:25:99:0|1:2348700_C_T:551,0,420:2348700"
977,chr1,3640485,.,ACCCGAGGCTCTGAGCATCAGCAGGGCGGGGTGCAGCG,A,1684.92,PASS,AC=4;AF=0.667;AN=6;DP=85;ExcessHet=0.0000;FS=0...,GT:AD:DP:GQ:PL,"0/0:19,0:19:48:0,48,720","1/1:0,25:25:79:1122,79,0","1/1:0,13:13:42:582,42,0"
1347,chr1,4420298,.,TG,"T,*",1113.52,PASS,"AC=3,1;AF=0.500,0.167;AN=6;DP=40;ExcessHet=0.0...",GT:AD:DP:GQ:PGT:PID:PL:PS,"0/0:3,0,0:3:3:.:.:0,3,45,3,45,45","1|2:0,15,9:24:99:0|1:4420298_TG_T:1008,378,333...","1|1:0,12,0:12:36:1|1:4420279_GTATATATATATATA_G..."
1431,chr1,4616480,.,G,GC,1521.12,PASS,AC=2;AF=0.333;AN=6;BaseQRankSum=0.391;DP=95;Ex...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0/0:21,0:21:60:.:.:0,60,900","0|1:14,23:37:99:0|1:4616480_G_GC:895,0,514:461...","0|1:20,17:37:99:0|1:4616480_G_GC:637,0,769:461..."
5690,chr1,15614193,.,CTTTTTTTTTTTT,C,1150.39,PASS,AC=2;AF=0.333;AN=6;BaseQRankSum=1.88;DP=61;Exc...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0/0:13,0:13:10:.:.:0,10,460","0|1:9,17:26:99:0|1:15614180_T_C:687,0,323:1561...","0|1:10,12:22:99:0|1:15614180_T_C:474,0,384:156..."
...,...,...,...,...,...,...,...,...,...,...,...,...
910061,chrX,141401950,.,GTA,G,1198.79,PASS,AC=4;AF=0.667;AN=6;DP=29;ExcessHet=0.0000;FS=0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0/0:2,0:2:3:.:.:0,3,45","1|1:0,15:15:45:1|1:141401882_ATATATATATGTGTATA...","1|1:0,12:12:36:1|1:141401882_ATATATATATGTGTATA..."
910063,chrX,141402038,.,GTATATATATATA,G,1591.97,PASS,AC=4;AF=0.667;AN=6;DP=44;ExcessHet=0.0000;FS=0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0/0:8,0:8:21:.:.:0,21,315","1|1:0,21:21:63:1|1:141401882_ATATATATATGTGTATA...","1|1:0,15:15:45:1|1:141401882_ATATATATATGTGTATA..."
910554,chrX,142844494,.,A,ACG,1018.10,PASS,AC=2;AF=0.333;AN=6;DP=56;ExcessHet=0.0000;FS=0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0/0:18,0:18:48:.:.:0,48,720","0/0:15,0:15:39:.:.:0,39,585","1|1:0,23:23:69:1|1:142844391_CATATATACGTAT_C:1..."
912790,chrX,152083224,.,AC,A,1369.92,PASS,AC=4;AF=0.667;AN=6;DP=60;ExcessHet=0.0000;FS=0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0/0:25,0:25:40:.:.:0,40,924","1|1:0,18:18:54:1|1:152083205_T_TA:727,54,0:152...","1|1:0,16:16:48:1|1:152083205_T_TA:662,48,0:152..."


In [13]:
'''
write the variants to a new file, keeping the header
'''
outfile=open('Donor0958_OR7A10OE_Unique_Indel_Variant.vcf', 'w')
for line in NK_header:
    outfile.write(line)
outfile.close()
OE_filtered.to_csv('Donor0958_OR7A10OE_Unique_Indel_Variant.vcf', mode='a', sep='\t', index=False)

outfile=open('Donor0958_OR7A10Stop_Unique_Indel_Variant.vcf', 'w')
for line in NK_header:
    outfile.write(line)
outfile.close()
Stop_filtered.to_csv('Donor0958_OR7A10Stop_Unique_Indel_Variant.vcf', mode='a', sep='\t', index=False)


